# Technical Factor - Average True Range (ATR)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        m_avg(GREATEST(ABS(high - low), ABS(m_lag(close, 1) - high), ABS(m_lag(close, 1) - low)), 15) AS ATR,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(ATR > m_lag(ATR, 1) AND m_lag(ATR, 1) >= m_lag(ATR, 2), 1, 0) AS TRBY1,
        IF(ATR < m_lag(ATR, 1) AND m_lag(ATR, 1) <= m_lag(ATR, 2), 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF((ATR - m_lag(ATR, 1)) > 0 AND ((ATR - m_lag(ATR, 1)) - (m_lag(ATR, 1) - m_lag(ATR, 2))) > 0, 1, 0) AS MTBY1,
        IF((ATR - m_lag(ATR, 1)) < 0 AND ((ATR - m_lag(ATR, 1)) - (m_lag(ATR, 1) - m_lag(ATR, 2))) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(ATR > m_lag(ATR, 1) AND (ATR - m_lag(ATR, 1)) < (m_lag(ATR, 1) - m_lag(ATR, 2)), 1, 0) AS RVBY1,
        IF(ATR < m_lag(ATR, 1) AND (ATR - m_lag(ATR, 1)) > (m_lag(ATR, 1) - m_lag(ATR, 2)), 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(ATR > m_avg(ATR, 15) AND m_lag(ATR, 1) <= m_avg(m_lag(ATR, 1), 15), 1, 0) AS BKBY1,
        IF(ATR < m_avg(ATR, 15) AND m_lag(ATR, 1) >= m_avg(m_lag(ATR, 1), 15), 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        ATR,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend    USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,ATR,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-22,600519.SH,223.436896,0,0,0,0,0,0,0,0
1,2025-01-23,600519.SH,215.567947,0,0,0,0,0,0,0,0
2,2025-01-24,600519.SH,209.865809,0,1,0,0,0,1,0,0
3,2025-01-27,600519.SH,196.153527,0,1,0,1,0,0,0,0
4,2025-02-05,600519.SH,211.304920,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
224,2025-12-25,600519.SH,125.498007,0,0,1,0,0,0,0,0
225,2025-12-26,600519.SH,123.120204,0,0,0,1,0,0,0,0
226,2025-12-29,600519.SH,120.176485,0,1,0,1,0,0,0,0
227,2025-12-30,600519.SH,117.586512,0,1,0,0,0,1,0,0
